In [1]:
import pandas as pd
import numpy as np
import requests as re
from bs4 import BeautifulSoup

import sys

In [2]:
sys.version

'3.10.5 (v3.10.5:f377153967, Jun  6 2022, 12:36:10) [Clang 13.0.0 (clang-1300.0.29.30)]'

In [3]:
wiki_films_url = 'https://en.wikipedia.org/wiki/List_of_A24_films'

page = re.get(url=wiki_films_url)
soup = BeautifulSoup(page.text, 'html.parser')

for h in soup.find_all('h3'):
    h.name = 'h2'

In [4]:
tables = soup.find_all('table')

In [5]:
table_data = []

In [6]:
for t in tables:
    
    page_section = t.find_previous('h2').text.removesuffix('[edit]')
    t_header = t.find_all('tr')[0]
    t_header_cols = [th.text.strip().removesuffix('[a]') for th in t_header.find_all('th')]
    
    data = {}
    
    for row_ix, row in enumerate(t.find_all('tr')[1:]):
        data[row_ix] = {}
        
        for th in row.find_all('th'):
            th.name = 'td'
                        
        for col_ix, col in enumerate(row.find_all('td')):

            data[row_ix][t_header_cols[col_ix]] = col.text.strip()
            
    table_data.append((page_section, data))


In [7]:
valid_sections = ['2010s', '2020s', 'Dated films'] 

In [8]:
final_df = pd.concat([pd.DataFrame.from_dict(t[1], orient='index') for t in table_data if t[0] in valid_sections], ignore_index=True)

In [9]:
final_df

,Release date,Title,Director(s),Synopsis,Notes,Ref.
0,"February 8, 2013",A Glimpse Inside the Mind of Charles Swan III,Roman Coppola,Charles' life falls apart when his girlfriend ...,,[4][6]
1,"March 15, 2013",Ginger & Rosa,Sally Potter,"In 1960s London, a teenager's best friend fall...",,[31][32]
2,"March 15, 2013",Spring Breakers,Harmony Korine,"During their spring break in Florida, four col...",,[33][34]
3,"June 14, 2013",The Bling Ring,Sofia Coppola,A group of fame-obsessed teenagers known as th...,,[35][36]
4,"August 2, 2013",The Spectacular Now,James Ponsoldt,An encounter between high schoolers Sutton and...,,[37][38]
...,...,...,...,...,...,...
149,"December 15, 2023",The Zone of Interest,Jonathan Glazer,The commandant of the Auschwitz concentration ...,,[315]
150,"December 22, 2023",The Iron Claw,Sean Durkin,The Von Erich family is a dynasty of wrestlers...,Also produced by A24,[316]
151,"December 25, 2023",Occupied City,Steve McQueen,A documentary set in present-day Amsterdam tha...,Also produced by A24,[317][318]
152,"March 8, 2024",Love Lies Bleeding,Rose Glass,Reclusive gym manager Lou falls hard for Jacki...,Also produced by A24,[319][320]


In [10]:
final_df['Release date'] = pd.to_datetime(final_df['Release date'])
final_df['Notes'] = final_df['Notes'].replace(r'^\s*$', np.nan, regex=True)
final_df = final_df.drop(['Ref.'], axis=1)

In [11]:
final_df

,Release date,Title,Director(s),Synopsis,Notes
0,2013-02-08,A Glimpse Inside the Mind of Charles Swan III,Roman Coppola,Charles' life falls apart when his girlfriend ...,NaN
1,2013-03-15,Ginger & Rosa,Sally Potter,"In 1960s London, a teenager's best friend fall...",NaN
2,2013-03-15,Spring Breakers,Harmony Korine,"During their spring break in Florida, four col...",NaN
3,2013-06-14,The Bling Ring,Sofia Coppola,A group of fame-obsessed teenagers known as th...,NaN
4,2013-08-02,The Spectacular Now,James Ponsoldt,An encounter between high schoolers Sutton and...,NaN
...,...,...,...,...,...
149,2023-12-15,The Zone of Interest,Jonathan Glazer,The commandant of the Auschwitz concentration ...,NaN
150,2023-12-22,The Iron Claw,Sean Durkin,The Von Erich family is a dynasty of wrestlers...,Also produced by A24
151,2023-12-25,Occupied City,Steve McQueen,A documentary set in present-day Amsterdam tha...,Also produced by A24
152,2024-03-08,Love Lies Bleeding,Rose Glass,Reclusive gym manager Lou falls hard for Jacki...,Also produced by A24


In [12]:
final_df[:50]

,Release date,Title,Director(s),Synopsis,Notes
0,2013-02-08,A Glimpse Inside the Mind of Charles Swan III,Roman Coppola,Charles' life falls apart when his girlfriend ...,NaN
1,2013-03-15,Ginger & Rosa,Sally Potter,"In 1960s London, a teenager's best friend fall...",NaN
2,2013-03-15,Spring Breakers,Harmony Korine,"During their spring break in Florida, four col...",NaN
3,2013-06-14,The Bling Ring,Sofia Coppola,A group of fame-obsessed teenagers known as th...,NaN
4,2013-08-02,The Spectacular Now,James Ponsoldt,An encounter between high schoolers Sutton and...,NaN
5,2014-03-14,Enemy,Denis Villeneuve,The lives of two identical men with two distin...,NaN
6,2014-04-04,Under the Skin,Jonathan Glazer,An otherworldly woman preys on men in Scotland...,NaN
7,2014-04-25,Locke,Steven Knight,Ivan Locke's life unravels as he answers sever...,NaN
8,2014-06-06,Obvious Child,Gillian Robespierre,"After a one-night stand, a pregnant stand-up c...",NaN
9,2014-06-13,The Rover,David Michôd,"Years after a global economic collapse, two me...",NaN
